In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('Datas/diabetes.csv')

In [3]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
import xgboost as xgb

In [5]:
from sklearn.cross_validation import train_test_split

C:\Users\user\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
from subprocess import check_output

In [7]:
data_full = data.copy()
X_data = data_full.drop('Outcome', axis = 1)
y = data_full.Outcome

In [8]:
X_data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [9]:
seed = 7
test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(X_data, y, test_size = test_size, random_state = seed)

In [10]:
# Train the XGBoost Model for Classification

In [11]:
model1 = xgb.XGBClassifier()
model2 = xgb.XGBClassifier(n_estimators = 100, max_depth = 8, learning_rate = 0.1, subsample = 0.5)

train_model1 = model1.fit(X_train, y_train)
train_model2 = model2.fit(X_train, y_train)

In [12]:
from sklearn.metrics import classification_report

pred1 = train_model1.predict(X_test)
pred2 = train_model2.predict(X_test)

print("Model 1 Xgboost report %r" % (classification_report(y_test, pred1)))
print("Model 2 Xgboost report %r" % (classification_report(y_test, pred2)))

Model 1 Xgboost report '             precision    recall  f1-score   support\n\n          0       0.77      0.82      0.79       147\n          1       0.64      0.58      0.61        84\n\navg / total       0.73      0.73      0.73       231\n'
Model 2 Xgboost report '             precision    recall  f1-score   support\n\n          0       0.78      0.79      0.79       147\n          1       0.63      0.62      0.62        84\n\navg / total       0.73      0.73      0.73       231\n'


C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [14]:
from sklearn.metrics import accuracy_score

print("Accuracy for model1: %.2f" % (accuracy_score(y_test, pred1) * 100))
print("Accuracy for model2: %.2f" % (accuracy_score(y_test, pred2) * 100))

Accuracy for model1: 73.16
Accuracy for model2: 72.73


In [20]:
model3 = xgb.XGBClassifier(
learning_rate = 0.1,
n_estimators = 1000,
max_depth = 5, 
min_child_weight = 1,
gamma = 0,
subsample = 0.8,
colsample_bytree = 0.8,
objective = 'binary : logistic',
nthread = 4, 
scale_pos_weight = 1,
seed = 27)

In [22]:
train_model3 = model3.fit(X_train, y_train)
pred3 = train_model3.predict(X_test)
print("Accuracy for model 3: %.2f" % (accuracy_score(y_test, pred3) * 100))

XGBoostError: [12:15:34] C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\objective\objective.cc:26: Unknown objective function: `binary : logistic`
Objective candidate: survival:aft
Objective candidate: binary:hinge
Objective candidate: multi:softmax
Objective candidate: multi:softprob
Objective candidate: rank:pairwise
Objective candidate: rank:ndcg
Objective candidate: rank:map
Objective candidate: reg:gamma
Objective candidate: reg:tweedie
Objective candidate: reg:squarederror
Objective candidate: reg:squaredlogerror
Objective candidate: reg:logistic
Objective candidate: reg:pseudohubererror
Objective candidate: binary:logistic
Objective candidate: binary:logitraw
Objective candidate: reg:linear
Objective candidate: count:poisson
Objective candidate: survival:cox


In [24]:
from sklearn.model_selection import GridSearchCV

param_test = {
    'max_depth': [4, 5, 6],
    'min_child_weight': [4, 5, 6]
}

gsearch = GridSearchCV(estimator = xgb.XGBClassifier(learning_rate = 0.1, n_estimators = 140, max_depth = 5, min_child_weight = 0.2, gamma = 0, subsample = 0.8, colsample_bytree = 0.8, 
objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

In [25]:
train_model4 = gsearch.fit(X_train, y_train)
pred4 = train_model4.predict(X_test)
print("Accuracy of model 4: %.2f " % (accuracy_score(y_test, pred4) * 100))

Accuracy of model 4: 73.59 


C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [27]:
param_test2b = {
    'min_child_weight': [6, 8, 10, 12]
}

gsearchb = GridSearchCV(estimator = xgb.XGBClassifier(learning_rate = 0.1, n_estimators = 14, max_depth = 4,
min_child_weight = 2, gamma = 0, subsample = 0.8, colsample_bytree = 0.8,
objective = 'binary:logistic', nthread = 4, scale_pos_weight = 1, seed = 27), param_grid = param_test2b, scoring = 'roc_auc', n_jobs = 4, iid = False, cv = 5)

train_model5 = gsearchb.fit(X_train, y_train)
pred5 = train_model5.predict(X_test)
print("Accuracy of model 5: %.2f " % (accuracy_score(y_test, pred5) * 100))

Accuracy of model 5: 77.06 


C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [29]:
param_test3 = {
    'gamma' : [i/10.0 for i in range (0, 5)]
}

gsearch3 = GridSearchCV(estimator = xgb.XGBClassifier(learning_rate = 0.1, n_estimators = 14, max_depth = 4,
min_child_weight = 2, gamma = 0, subsample = 0.8, colsample_bytree = 0.8,
objective='binary:logistic', nthread = 4, scale_pos_weight = 1, seed = 27), param_grid = param_test3, scoring = 'roc_auc', n_jobs = 4, iid = False, cv = 5 )

train_model6 = gsearch3.fit(X_train, y_train)
pred6 = train_model6.predict(X_test)
print("Accuracy of model 6: %.2f" % (accuracy_score(y_test, pred6) * 100))

Accuracy of model 6: 74.89


C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [30]:
xgb = xgb.XGBClassifier(
learning_rate = 0.7,
n_estimators = 1000,
max_depth = 4, 
min_child_weight = 6,
gamma = 0,
subsample = 0.8,
colsample_bytree = 0.8,
objective = 'binary : logistic',
nthread = 4, 
scale_pos_weight = 1,
seed = 27)

train_model7 = xgb.fit(X_train, y_train)
pred7 = train_model7.predict(X_test)
print("Accuracy of model 7: %.2f" % (accuracy_score(y_test, pred7) * 100))

XGBoostError: [13:17:05] C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\objective\objective.cc:26: Unknown objective function: `binary : logistic`
Objective candidate: survival:aft
Objective candidate: binary:hinge
Objective candidate: multi:softmax
Objective candidate: multi:softprob
Objective candidate: rank:pairwise
Objective candidate: rank:ndcg
Objective candidate: rank:map
Objective candidate: reg:gamma
Objective candidate: reg:tweedie
Objective candidate: reg:squarederror
Objective candidate: reg:squaredlogerror
Objective candidate: reg:logistic
Objective candidate: reg:pseudohubererror
Objective candidate: binary:logistic
Objective candidate: binary:logitraw
Objective candidate: reg:linear
Objective candidate: count:poisson
Objective candidate: survival:cox


In [34]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc_model = rfc.fit(X_train, y_train)
pred8 = rfc_model.predict(X_test)
print("Accuracy of Random Forest Model: %.2f" % (accuracy_score(y_test, pred8) * 100))

Accuracy of Random Forest Model: 74.03
